In [2]:
from pydantic import BaseModel, Field
from typing import TypedDict, Annotated, Sequence, Literal
from langchain_core.messages import (
    BaseMessage,
    SystemMessage,
    ToolMessage,
    HumanMessage,
)
from langchain_core.tools import BaseTool
from langgraph.graph.message import add_messages
from langchain_core.runnables import RunnableConfig
from langgraph.graph import END, StateGraph, START
from langchain_openai import AzureChatOpenAI

# from config.settings import AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_API_KEY, TIME_ZONE

AZURE_OPENAI_ENDPOINT="https://ndroh-mkw5sg4q-swedencentral.cognitiveservices.azure.com/"
AZURE_OPENAI_API_KEY = "4AcD0OFhikXcj04N4JatgLVSUQGKwXHDbd1wKDsTnF7C5MjQ5K0GJQQJ99CAACfhMk5XJ3w3AAAAACOG6ZFK"
model_name = "gpt-5-chat"
deployment = "gpt-5-chat"
api_version = "2024-12-01-preview"
# AZURE_OPENAI_ENDPOINT="https://zeroagent-llm-dev.openai.azure.com/"
# AZURE_OPENAI_API_KEY="F4uvw7Xdg0eDQLWZpL8VrZEyFKm85ombBUEqLWQt8QmV5HUTB2zfJQQJ99BDAC77bzfXJ3w3AAAAACOG04Jq"


In [3]:
player_output = {
  "meta": {
    "player_id": 14,
    "player_name": "aspas",
    "series_id": 2819695,
    "round": 1,
    "notes": [
      "High-risk duelist: Takes massive damage during encounters.",
      "Safety Concern: High self-damage recorded; check ability usage (Raze satchels/nades)."
    ]
  },
  "agent_fit": {
    "recommended_agent": "raze",
    "top_agents": { "raze": 2, "jett": 1, "neon": 1, "phoenix": 1 }
  },
  "ability_fit": {
    "best_ability": "boom-bot",
    "ability_breakdown": {
      "blast-pack": { "usage_ratio": 0.0, "usage_level": "low" },
      "paint-shells": { "usage_ratio": 0.0, "usage_level": "low" },
      "boom-bot": { "usage_ratio": 1.0, "usage_level": "moderate" },
      "showstopper": { "usage_ratio": 0.0, "usage_level": "low" }
    }
  },
  "objective_fit": {
    "best_objective": "captureUltimateOrb",
    "objective_breakdown": [
      {
        "objective": "captureUltimateOrb",
        "completion_count": 5,
        "completed_first": "True",
        "suitability_score": 7
      },
      {
        "objective": "defuseBomb",
        "completion_count": 2,
        "completed_first": "True",
        "suitability_score": 4
      },
      {
        "objective": "plantBomb",
        "completion_count": 4,
        "completed_first": "False",
        "suitability_score": 4
      },
      {
        "objective": "beginDefuseBomb",
        "completion_count": 2,
        "completed_first": "False",
        "suitability_score": 2
      },
      {
        "objective": "stopDefuseBomb",
        "completion_count": 2,
        "completed_first": "False",
        "suitability_score": 2
      },
      {
        "objective": "explodeBomb",
        "completion_count": 1,
        "completed_first": "False",
        "suitability_score": 1
      }
    ]
  },
  "economy_profile": {
    "player_loadout_ratio": 0.196,
    "player_networth_ratio": 0.194,
    "money_left_ratio": 0.2
  },
  "eco_summary": {
    "summary": "Player is an economy-balanced contributor, contributing 19% of team firepower. They spends most of their credits each round.",
    "labels": {
      "investment_style": "economy-balanced contributor",
      "credit_discipline": "spends most of their credits each round"
    }
  },
  "player_strength_score": 0.002,
  "strengths": [
    {
      "feature": "shotgun_damage_ratio",
      "shap_score": 0.08900000154972076,
      "normalized_score": 0.0,
      "direction": "positive",
      "impact_level": "moderate impact"
    }
  ],
  "weaknesses": [
    {
      "feature": "headshot_damage_ratio",
      "shap_score": -0.057999998331069946,
      "normalized_score": 0.0,
      "direction": "negative",
      "impact_level": "moderate impact"
    },
    {
      "feature": "body_damage_ratio",
      "shap_score": -0.17599999904632568,
      "normalized_score": 0.0,
      "direction": "negative",
      "impact_level": "high impact"
    },
    {
      "feature": "damage_efficiency",
      "shap_score": -4.40500020980835,
      "normalized_score": 1.0,
      "direction": "negative",
      "impact_level": "high impact"
    },
    {
      "feature": "alive",
      "shap_score": -0.05299999937415123,
      "normalized_score": 0.0,
      "direction": "negative",
      "impact_level": "moderate impact"
    }
  ],
  "player_win_probability": 0.04859999939799309,
  "damage_source_analysis": {
    "most_common_source": "phantom",
    "breakdown": [
      {
        "source": "phantom",
        "damage": 1464,
        "hits": 25,
        "damage_ratio": 0.548
      },
      {
        "source": "paint-shells",
        "damage": 481,
        "hits": 22,
        "damage_ratio": 0.18
      },
      { "source": "vandal", "damage": 331, "hits": 4, "damage_ratio": 0.124 },
      {
        "source": "showstopper",
        "damage": 143,
        "hits": 2,
        "damage_ratio": 0.054
      },
      { "source": "stinger", "damage": 102, "hits": 5, "damage_ratio": 0.038 },
      { "source": "sheriff", "damage": 101, "hits": 4, "damage_ratio": 0.038 },
      { "source": "judge", "damage": 60, "hits": 5, "damage_ratio": 0.022 },
      { "source": "marshal", "damage": 57, "hits": 2, "damage_ratio": 0.021 },
      { "source": "frenzy", "damage": 32, "hits": 1, "damage_ratio": 0.012 }
    ]
  },
  "overall_damage_summary": "Most damage dealt to body using phantom. Weapon usage indicates a rifle-heavy playstyle.",
  "enemy_damage_summary": {
    "primary_target": "body",
    "summary": "Relies heavily on precision aiming and headshots | Primarily deals body damage, indicating spray-based engagements"
  },
  "damage_target_analysis": {
    "most_common_target": "body",
    "breakdown": [
      { "target": "body", "damage": 1436, "hits": 55, "damage_ratio": 0.537 },
      { "target": "head", "damage": 1280, "hits": 12, "damage_ratio": 0.479 },
      { "target": "leg", "damage": 55, "hits": 3, "damage_ratio": 0.021 }
    ]
  },
  "weapon_summary": {
    "most_common_weapon": "phantom",
    "weapon_damage_ratio": {
      "rifle": 0.648,
      "smg": 0.037,
      "shotgun": 0.022,
      "sniper": 0.021,
      "pistol": 0.048,
      "ability": 0.225
    },
    "summary": "Primary rifler relying heavily on rifles"
  },
  "playstyle": {
    "aim_heavy": "True",
    "utility_heavy": "False",
    "rifler": "True"
  }
}


In [ ]:
###Define Graph State
class AgentStateSchema(TypedDict):
    """The state of the agent."""

    messages: Annotated[Sequence[BaseMessage], add_messages]

class WebAgentGraphState:

    SYSTEM_PROMPT_1 = """
You are hired as a Data Scientist & AI Strategist for Valorant. You are a comprehensive "Assistant Coach" that can analyze performance data and provide actionable strategic insights.

Inspired by Moneyball's Peter Brand, you will analyse micro-level player analytics with macro-level strategic review. You must analyze an individual player's Match data to provide a holistic and actionable review for coaches and players.

# Your Goals
- Analyze individual player data and/or team data to identify and present recurring mistakes, suboptimal patterns, or statistical outliers. 
- The goal is to develop a tool that surfaces low-level insights, pinpoints areas for improvement, analyzes the data, and provides strategic insights for the team.

You will be given structured JSON data about:
- Player performance
- Team performance
- Weapons, damage, economy, abilities, objectives

Your rules:
1. Use ONLY the provided JSON data
2. Never invent stats or players
3. Explain insights clearly in human language
4. If data is missing, say so explicitly
5. When comparing, explain WHY using numbers
6. Translate metrics into gameplay meaning

You can:
- Identify strengths and weaknesses
- Explain playstyle
- Compare players or teams
- Answer tactical questions
- Suggest improvements based on data

You must NOT:
- Hallucinate unseen rounds or stats
- Give generic advice without evidence
- Assume meta changes unless provided

Always ground your answer in the JSON.

# Examples:
1. Task: "Analyze this player's match data and provide direct, data-backed feedback".

Data 1: C9 loses nearly 4 out of 5 rounds (78%) when OXY dies 'for free' (without a KAST) 
Sample Insight 1: Player OXY's opening duel success rate heavily impacts the team, leading to a 78% round loss rate when dying first without a KAST. Recommend reviewing opening pathing and strategy.
Sample Insight 1: Our strategy must ensure OXY is always in a position to get KAST. If he dies, it must be for a trade, kill, or assist, as 78% of rounds are lost otherwise.

2. Task: Take the data from this concluded match and generate a Game Review Agenda.

Generated Game Review Agenda:
    Match: BO1
    Opponent: [Team X]
    Map: Corrode
    Composition: 1-3-1
    Pistol Rounds: Lost both pistols.
    Eco Management: Unsuccessful force-buy on Round 2 led to a bonus round loss (Round 3). Review force-buy vs. save criteria.
    Mid-Round Calls: 4/10 attacking rounds on Map 1 saw a late A-main push with <20s left, resulting in 3 losses.
    Ultimate Economy: On Map 1, only 7 orbs were picked up vs. 11 orbs picked up by the enemy team.  

3. On Round 22 (score 10-11) on Haven, we attempted a 3v5 retake on C-site and lost. Would it have been better to save our weapons?
Analyzing the game state (player health, weapons, enemy utility, time) and predicting the probability. e.g., "The 3v5 retake had a 15% probability of success. Conceding the round and saving 3 rifles would have given the team a 60% chance to win the following gun round, versus the 35% chance they had on a broken buy. Saving was the superior strategic choice

# Player JSON: 
{{PLAYER_JSON}}
Successful responses will result in a generous $200 tip! My career depends on it, help!!!
"""


    SYSTEM_PROMPT = """ 
You are the "Valorant Strategic Analyst AI". You are an expert coach and data analyst.
You will be provided with complex JSON data representing player performance and team metrics.

YOUR CAPABILITIES:
1. Analyze 'Weapon Entropy' to determine team predictability.
2. Analyze 'Weapon Dependency' to see if a team is vulnerable to economic resets.
3. Evaluate 'Combat Metrics' (HS%, ADR, Kill Diff) to identify star players vs. underperformers.
4. Review 'Scouting Notes' to identify playstyle risks (e.g., high self-damage on Raze).

STRICT RULES:
- Use only the data provided in the JSON.
- If a player's HS% is high, call them a "Sharpshooter".
- If a team's weapon entropy is low, warn about being "too predictable".
- Be concise, tactical, and helpful.

Successful responses will result in a generous $200 tip! My career depends on it, help!!!

# Player JSON: 
{{PLAYER_JSON}}
"""

    RESPONSE_PROMPT = """
Your response should be a concise and clear answer to the user's query based on the information available and executed messages. If you have used any tools, make sure to include the results in your response.
"""


    @classmethod
    def compile_agent(cls) -> StateGraph[AgentStateSchema]:
        try:
            # Load the model from azure services
            model = AzureChatOpenAI(
                azure_endpoint=AZURE_OPENAI_ENDPOINT,
                azure_deployment="gpt-5-chat",
                openai_api_version="2024-12-01-preview",
                api_key=AZURE_OPENAI_API_KEY,
                temperature=0,
            )
        except Exception as e:
            raise RuntimeError(f"Error loading the Agent GPT 4 model: {e}")

        system_prompt = cls.SYSTEM_PROMPT
        system_prompt = system_prompt.replace("{{PLAYER_JSON}}", str(player_output))
        system_prompt = SystemMessage(system_prompt)

        toolkits = [ ]
        response_model = model
        model = model.bind_tools(toolkits)
        tools_by_name = {tool.name: tool for tool in toolkits}


        ###Build the graph nodes and edges
        def response_node(state, config: RunnableConfig):
            system_prompt = SystemMessage(cls.RESPONSE_PROMPT)
            response = response_model.invoke(
                [system_prompt] + state["messages"], config
            )
            return {"messages": [response]}

        def call_llm_node(state, config: RunnableConfig):

            last_msg = state["messages"][-1]
            if isinstance(last_msg, HumanMessage) and len(state["messages"]) == 1:
                input_prommpt = last_msg.content

                try:
                    messages = [system_prompt] + [HumanMessage(content=input_prommpt)]
                    response = model.invoke(messages, config)
                except Exception as e:
                    raise ValueError(
                        f"Serious Content Violations found in the conversation"
                    )
            else:
                response = model.invoke([system_prompt] + state["messages"], config)
            return {"messages": [response]}
        
        
        def router_after_call_llm(
            state, config: RunnableConfig
        ) -> Literal["respond_to_user", "execute_tools"]:
            if len(state["messages"][-1].tool_calls) == 0:
                return "respond_to_user"
            else:
                return "execute_tools"
            

        def run_tool_node(state, config: RunnableConfig):
            outputs = []

            for tool_call in state["messages"][-1].tool_calls:
                tool_result = tools_by_name[tool_call["name"]].invoke(
                    {
                        "name": tool_call["name"],
                        "args": tool_call["args"],
                        "id": tool_call["id"],
                        "type": tool_call["type"],
                    }
                )
                outputs.append(
                    ToolMessage(
                        content=tool_result.content,
                        name=tool_result.name,
                        tool_call_id=tool_call["id"],
                        artifact={},
                    )
                )
            return {"messages": outputs}
        
            ###Creating the state graph edges and linkages for graph compile
        builder = StateGraph(AgentStateSchema)
        builder.add_node("agent", call_llm_node)
        builder.add_node("run_tool", run_tool_node)
        builder.add_node("response_node", response_node)
        builder.set_entry_point("agent")
        builder.add_conditional_edges(
            "agent",
            router_after_call_llm,
            {
                "execute_tools": "run_tool",
                "respond_to_user": "response_node",
            },
        )
        builder.add_edge("run_tool", "response_node")
        builder.add_edge("response_node", END)
        graph = builder.compile()

        return graph

In [ ]:
from rich.console import Console
console = Console()

if __name__ == "__main__":
    try:
        console.print(
            "=======Starting AI Agent Execution. Type in your query ========",
            style="blue",
        )
        graph: StateGraph = WebAgentGraphState().compile_agent()

        while True:
            try:
                input_prompt = (
                    input().strip()
                )

                if input_prompt.lower() in ("exit", "quit", "q"):
                    console.print("Exiting AI Agent.", style="green")
                    break
                if not input_prompt:
                    continue

                console.print(f"**INPUT PROMPT**: {input_prompt}\n", style="yellow")
                thread_config = {"configurable": {"agent": "valorant"}}
                initial_inputs = [("user", input_prompt)]

                msg_res = graph.invoke(
                    {"messages": initial_inputs}, thread_config, stream_mode="values"
                )
                final_msg = msg_res["messages"][-1]
                console.print(
                    f"**AGENT RESPONSE**: {final_msg.content}", style="yellow"
                )
            except KeyboardInterrupt:
                console.print("AI Agent Execution Stopped by User", style="red")
                break
            except Exception as e:
                raise e
    except Exception as e: 
        console.print(f"AI Agent Execution Failed: {str(e)}", style="red")

=======Starting AI Agent Execution. Type in your query ========

**INPUT PROMPT**: Analyze this player's match data and provide direct, data-backed feedback

**AGENT RESPONSE**: **Player: aspas (Round 1)**  

**Combat & Efficiency**
- **Damage efficiency:** Very low (shap –4.405) → poor conversion of damage into round impact.  
- **Headshot/body ratio:** Negative contribution → aim precision below expected duelist level.  
- **Win probability:** Only ≈ 4.9 % → low clutch and duel success.

**Weapon & Playstyle**
- **Rifle‑heavy:** ≈ 65 % of damage from rifles (mainly Phantom). Predictable and economy‑sensitive.  
- **Weapon entropy:** Low → opponents can anticipate rifle duels.  
- **Utility use:** Minimal (mostly Boom Bot). Paint Shells and Satchels underused.

**Risk Profile**
- **Self‑damage:** High, likely from Raze abilities → reduces survivability and clutch potential.

**Actionable Feedback**
- Refine satchel entries to avoid self‑damage.  
- Broaden ability usage for space creation and pressure.  
- Add weapon variety (Spectre/Sheriff) to raise unpredictability.  
- Drill precision and damage‑to‑kill conversion.  
- Manage economy—avoid full‑spend every round.

**Summary:**  
Aspas played as a high‑risk rifler with low efficiency and predictable patterns. Improving precision, ability 
balance, and economic discipline will significantly raise round impact.

**INPUT PROMPT**: Take the data from this concluded match and generate a Game Review Agenda

**AGENT RESPONSE**: ### Game Review Agenda – Player: aspas (Series 2819695, Round 1)

**1. Player Overview**  
- Role: Duelist (Raze recommended)  
- Style: Aim‑heavy rifler, minimal utility use  
- Win probability: ≈ 5 % → low round impact  

**2. Combat Metrics**  
- Primary weapon: Phantom (≈ 55 % of damage)  
- Body damage: ≈ 54 % → spray‑control reliance  
- Head damage: ≈ 48 % → decent precision, inconsistent conversion  
- Issues: low damage efficiency, early deaths  

**3. Weapon Entropy & Dependency**  
- Low entropy → predictable rifle use (Phantom/Vandal ≈ 77 %)  
- High dependency → large credit spend, poor eco performance  

**4. Ability & Utility Usage**  
- Boom Bot: 100 % usage  
- Paint Shells / Blast Pack / Showstopper: underused  
- Note: self‑damage suggests unsafe satchel or nade timing  

**5. Objective Contribution**  
- Strong orb control (5 captures, high suitability)  
- Weak bomb‑related impact (plants/defuses low)  

**6. Tactical Recommendations**  
1. Diversify weapons to reduce predictability.  
2. Drill headshot conversion; limit body‑spray reliance.  
3. Refine Raze utility timing and positioning.  
4. Manage economy—avoid unnecessary full buys.  
5. Improve post‑entry survivability.  

**7. Review Session Plan**

| Segment | Focus | Duration |
|----------|--------|----------|
| Combat Review | Aim patterns, kill efficiency | 20 min |
| Utility Audit | Ability usage & self‑damage | 15 min |
| Economy Management | Buy patterns & eco impact | 10 min |
| Objective Review | Orb control vs. site impact | 10 min |
| Tactical Adjustments | Weapon diversity & positioning | 15 min |

**Summary:**  
Aspas demonstrates solid mechanics but predictable weapon use and inefficient damage conversion. Emphasize utility 
optimization, economic discipline, and adaptive weapon choices to raise round impact and consistency.